In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):

        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
import random
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [5]:
import pandas as pd

In [6]:
df =pd.read_csv("reviews.csv", sep='\t')
df = df[['review_id', 'app_id', 'review_text', 'label']]
df.to_csv('reviews.csv', sep='\t', index=False)
df.shape

(40129, 4)

In [7]:
df.groupby('label').count()

,review_id,app_id,review_text
label,,,
0.0,8109,8109,8107
1.0,32019,32019,31966


In [8]:
df.label.mean()

0.7979216507177034

# build model

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, HashingTF
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import expr
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import Word2Vec
from pyspark.ml import Pipeline
from pyspark.sql.functions import split
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

file_location="reviews.csv"
#text_df = spark.read.text(file_location)

In [47]:
# create a SparkSession 
spark = SparkSession.builder \
    .appName("assignment 3") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.cores", "2") \
    .getOrCreate()

In [11]:
#text_df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine", "true").csv(file_location)

# read csv

text_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiline", "true") \
    .option("delimiter", "\t") \
    .csv(file_location)

text_df = text_df.select(col('review_text'), col('label'))
text_df = text_df.dropna()
text_df.show()

+--------------------+-----+
|         review_text|label|
+--------------------+-----+
|i can confirm tha...|  1.0|
|Really good game,...|  1.0|
|Its not finished ...|  1.0|
|Hey. It's really ...|  1.0|
|          Fun so far|  1.0|
|I'd rather play W...|  0.0|
|I have been playi...|  1.0|
|Nice game! Loads ...|  1.0|
|All hail NA serve...|  1.0|
|enjoying it so fa...|  1.0|
|This game came as...|  1.0|
|         Great fun  |  1.0|
|Fantastic consept...|  1.0|
|You know the game...|  1.0|
|EDIT: My issues h...|  1.0|
|AMAZIIIIIIIIIIING...|  1.0|
|In its current st...|  0.0|
|Играю в Калибр го...|  1.0|
|Secret Word: Prou...|  1.0|
|Fated Word: Death...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [12]:
# Get the number of rows and columns in the DataFrame
num_rows = text_df.count()
num_cols = len(text_df.columns)

# Print the shape of the DataFrame
print("Shape of the DataFrame: (%d, %d)" % (num_rows, num_cols))

Shape of the DataFrame: (38890, 2)


In [13]:
# Remove duplicate rows
text_df = text_df.dropDuplicates()
# Remove rows with missing values
text_df = text_df.na.drop()

In [14]:
text_df.select("label").distinct().show()

+--------------------+
|               label|
+--------------------+
|                 1.0|
|                 0.0|
|It's been at leas...|
|[td]✔️ Exciting p...|
|- I literally onl...|
+--------------------+



In [15]:
# Remove rows with unexpected labels
text_df = text_df.filter((col("label") == 1.0) | (col("label") == 0.0))
print("Number of rows after filtering: ", text_df.count())

Number of rows after filtering:  36913


In [16]:
train_data, test_data = text_df.limit(15014).randomSplit([0.8, 0.2], seed=7)
print("1 labelled: ", train_data.filter(col("label") == 1.0).count()+test_data.filter(col("label") == 1.0).count())
print("0 labelled: ", train_data.filter(col("label") == 0.0).count()+test_data.filter(col("label") == 0.0).count())

1 labelled:  11926
0 labelled:  3088


In [19]:
# preprocess the data
tokenizer = Tokenizer(inputCol="review_text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", locale="en_US")
count_vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")
string_indexer = StringIndexer(inputCol="label", outputCol="label_index")

# create model
lr = LogisticRegression(featuresCol="features", labelCol="label_index")

In [20]:
# define params grid
param_grid = ParamGridBuilder() \
   .addGrid(count_vectorizer.vocabSize, [1000, 5000]) \
   .addGrid(lr.regParam, [0.01, 0.1]) \
   .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
   .build()

# define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction")

In [21]:
# create pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, string_indexer, lr])

# define the cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# fit pipeline to the training data
cv_model = cv.fit(train_data)

# make predictions on the test data
predictions = cv_model.transform(test_data)

In [22]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction")

# Calculate accuracy
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

# Calculate F1 score
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

# Calculate recall
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

# Calculate precision
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})

print("Accuracy: {:.4f}".format(accuracy))
print("F1 score: {:.4f}".format(f1_score))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))


Accuracy: 0.8657
F1 score: 0.8573
Recall: 0.8657
Precision: 0.8576


In [23]:
from pyspark.mllib.evaluation import MulticlassMetrics

# Convert the predictions and labels to an RDD
predictionAndLabels = predictions.select("prediction", "label_index").rdd.map(lambda r: (r[0], r[1]))

# Instantiate a MulticlassMetrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Get the confusion matrix as a NumPy array
confusion_matrix = metrics.confusionMatrix().toArray()

# Print the confusion matrix
print("Confusion matrix:")
print(confusion_matrix)

C:\spark\spark-3.3.2-bin-hadoop2\python\pyspark\sql\context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion matrix:
[[2321.  118.]
 [ 296.  348.]]


In [24]:
# Get the best model from the cross-validation process
best_model = cv_model.bestModel

# Save the my_model
best_model.save("my_logistic_regression3")


In [46]:
lr_stage = best_model.stages[-1] 
print("Best Parameters: ")
print("Vocab Size: ", best_model.stages[2].getVocabSize())  # Assuming count_vectorizer is at index 2
print("Regularization Parameter: ", lr_stage.getRegParam())
print("Elastic Net Parameter: ", lr_stage.getElasticNetParam())

Best Parameters: 
Vocab Size:  5000
Regularization Parameter:  0.01
Elastic Net Parameter:  0.0


In [48]:

from pyspark.ml import PipelineModel
globals()['models_loaded'] = False
globals()['my_model'] = None

global results
results = []

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df = df.withColumn("label", col("label").cast("float"))
    df.show()
    
#     # Utilize our predict function
#     df_withpreds = df.withColumn("pred", predict_udf(
#         struct([df[x] for x in df.columns])
#     ))
#     df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = PipelineModel.load('my_logistic_regression3')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model (uncomment below):
    
    df_result = globals()['my_model'].transform(df)
    df_result.select('label', 'review_text', 'prediction','probability', 'label_index').show()
    
    collected_results = df_result.select('prediction', 'label_index').collect()
    results.extend(collected_results)
    
    # If we have collected 10 results, show the data and clear the results list
    display_results(results)


def display_results(results):
    result_df = spark.createDataFrame(results)
    predictionAndLabels = result_df.select("prediction", "label_index").rdd.map(lambda r: (r[0], r[1]))
    metrics = MulticlassMetrics(predictionAndLabels)
    confusion_matrix = metrics.confusionMatrix().toArray()
    print("Confusion matrix:")
    print(confusion_matrix)

In [49]:
ssc = StreamingContext(sc, 10)

In [50]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [51]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2023-05-27 10:18:10 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1934780|  0.0|139062886|i have 60000 ping...|
|1934780|  1.0|139062252|                  OH|
|1304930|  1.0|139064095|I am a very big f...|
+-------+-----+---------+--------------------+

+-----+--------------------+----------+--------------------+-----------+
|label|         review_text|prediction|         probability|label_index|
+-----+--------------------+----------+--------------------+-----------+
|  0.0|i have 60000 ping...|       1.0|[0.34555838361673...|        1.0|
|  1.0|                  OH|       0.0|[0.86250050155218...|        0.0|
|  1.0|I am a very big f...|       0.0|[0.99999551381068...|        0.0|
+-----+--------------------+----------+--------------------+-----------+

Confusion matrix:
[[2. 0.]
 [0. 1.]]
========= 2023-05-27 10:18:20 =========
+-------+-----+---------+----------------

+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2005010|  1.0|139063946|MY ARMOUR IS CONT...|
|2005010|  1.0|139063940|20 dollah make yo...|
+-------+-----+---------+--------------------+

+-----+--------------------+----------+--------------------+-----------+
|label|         review_text|prediction|         probability|label_index|
+-----+--------------------+----------+--------------------+-----------+
|  1.0|MY ARMOUR IS CONT...|       0.0|[0.85303644236814...|        0.0|
|  1.0|20 dollah make yo...|       0.0|[0.86712068851988...|        0.0|
+-----+--------------------+----------+--------------------+-----------+

Confusion matrix:
[[21.  2.]
 [ 3.  2.]]
========= 2023-05-27 10:20:10 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2005010|  1.0|139063879|Absolutely love t...|
|2005010|

Confusion matrix:
[[38.  4.]
 [ 5.  3.]]
========= 2023-05-27 10:22:00 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2419330|  1.0|139063166|Nice game 👍🏽 Fu...|
+-------+-----+---------+--------------------+

+-----+--------------------+----------+--------------------+-----------+
|label|         review_text|prediction|         probability|label_index|
+-----+--------------------+----------+--------------------+-----------+
|  1.0|Nice game 👍🏽 Fu...|       0.0|[0.97680649839410...|        0.0|
+-----+--------------------+----------+--------------------+-----------+

Confusion matrix:
[[39.  4.]
 [ 5.  3.]]
========= 2023-05-27 10:22:10 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2219690|  1.0|139063806|I would add this ...|
| 855740|  0.0|139066060|Piece of garbage....|
+----

+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1304930|  1.0|139066058|Fun game, exited ...|
|1304930|  1.0|139066053|Awesome game with...|
|1304930|  1.0|139066012|                hapy|
|1304930|  1.0|139065902|     i shat my pants|
|1304930|  1.0|139065861|Very fun with fri...|
+-------+-----+---------+--------------------+

+-----+--------------------+----------+--------------------+-----------+
|label|         review_text|prediction|         probability|label_index|
+-----+--------------------+----------+--------------------+-----------+
|  1.0|Fun game, exited ...|       0.0|[0.96367204201660...|        0.0|
|  1.0|Awesome game with...|       0.0|[0.99917718474980...|        0.0|
|  1.0|                hapy|       0.0|[0.85303644236814...|        0.0|
|  1.0|     i shat my pants|       0.0|[0.90045771430189...|        0.0|
|  1.0|Very fun with fri...|       0.0|[0.96259663437834...|     

In [52]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
